In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.4 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

imdb = load_dataset('imdb')

imdb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)



Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tokenizing_fn(example):
    
    return tokenizer(example['text'], truncation=True, padding=True)

tokenized_imdb = imdb.map(tokenizing_fn, batched=True)



  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [6]:
from transformers import DataCollatorWithPadding, TFAutoModelForSequenceClassification, create_optimizer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)




Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [9]:
import tensorflow as tf
from tensorflow import keras

In [8]:
tokenizer_columns = tokenizer.model_input_names
tokenizer_columns

['input_ids', 'attention_mask']

In [11]:
batch_size=8
num_epochs = 2
num_train_step = len(tokenized_imdb['train'])*num_epochs
lr_schedule = keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=5e-5, end_learning_rate=0.0,decay_steps=num_train_step)

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [14]:
tf_train_dataset = tokenized_imdb["train"].to_tf_dataset(columns=tokenizer_columns,
                                                         label_cols=['label'],
                                                         shuffle=True,
                                                          batch_size=batch_size,
                                                          collate_fn = data_collator,
                                                         )

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
tf_validation_dataset = tokenized_imdb["test"].to_tf_dataset(columns=tokenizer_columns,
                                                         label_cols=['label'],
                                                         shuffle=False,
                                                          batch_size=batch_size,
                                                          collate_fn = data_collator,
                                                         )

In [17]:
adam = keras.optimizers.Adam(learning_rate=lr_schedule)

In [18]:
model.compile(optimizer=adam, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [19]:
history = model.fit(x=tf_train_dataset, validation_data=tf_validation_dataset, epochs=2)

Epoch 1/2
3125/3125 [==============================] - 2006s 638ms/step - loss: 0.2802 - accuracy: 0.8862 - val_loss: 0.2139 - val_accuracy: 0.9135
Epoch 2/2
3125/3125 [==============================] - 1994s 638ms/step - loss: 0.1488 - accuracy: 0.9445 - val_loss: 0.2300 - val_accuracy: 0.9149


In [ ]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(16,), dtype=tf.int64, name=None))>

In [ ]:
tf_valid_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>